# Concepts

```{note}
The package is in the alpha phase of development. This notebook illustrates the concepts and a current state of development. 
Anything you see here may, and likely will, change.
```

The goal is to mimic what stars package in R can do - [https://r-spatial.github.io/stars/articles/stars1.html#vector-data-cube-example](https://r-spatial.github.io/stars/articles/stars1.html#vector-data-cube-example).

![vector data cube illustration](https://raw.githubusercontent.com/r-spatial/stars/master/images/cube3.png)

Because we need to use geometries as an index, we need to ensure that geopandas is using shapely 2.0 (beta) as a geometry engine. Shapely 1.8 geometries are not hashable, while shapely 2.0 are.

In [1]:
import geopandas
import shapely
import pandas
import xarray
import numpy

from xvec import GeometryIndex

We want to recreate [this example from stars documentation](https://r-spatial.github.io/stars/articles/stars1.html#vector-data-cube-example).

Load geometries using geopandas:

In [2]:
nc = geopandas.read_file("https://github.com/r-spatial/sf/raw/main/inst/gpkg/nc.gpkg")

Get the GeometryArray. Note that this also contains CRS information at this point.

In [3]:
origin = destination = nc.geometry.array

Create dimensions and dummy data.

In [4]:
mode = ["car", "bike", "foot"]
day = pandas.date_range("2015-01-01", periods=100)
hours = range(24)
data = numpy.random.randint(1, 100, size=(3, 100, 24, 100, 100))

Create `xarray.DataArray`.

In [5]:
arr = xarray.DataArray(data, coords=(mode, day, hours, origin, destination), dims=["mode", "day", "time", "origin", "destination"])
arr

<xarray.DataArray (mode: 3, day: 100, time: 24, origin: 100, destination: 100)>
array([[[[[93, 38,  9, ...,  4, 45, 31],
          [49, 62, 40, ...,  5,  9, 59],
          [26, 14, 86, ..., 36, 55, 70],
          ...,
          [63, 22, 82, ..., 28, 16, 27],
          [51, 52, 33, ..., 79, 51, 57],
          [34, 19, 14, ...,  6, 25, 67]],

         [[10, 48, 23, ..., 95, 97,  3],
          [67, 99, 47, ..., 26, 82, 10],
          [66, 49,  3, ..., 11, 79, 32],
          ...,
          [79, 55, 51, ..., 16, 95, 83],
          [50,  1, 75, ..., 57, 34, 30],
          [32, 44, 56, ..., 12, 42, 96]],

         [[25, 65, 47, ..., 72, 83, 55],
          [ 5, 97, 93, ..., 60,  7, 98],
          [58, 44, 38, ..., 27, 72, 14],
          ...,
...
          ...,
          [23, 19, 82, ..., 91,  4, 90],
          [ 7, 86, 98, ..., 81, 13, 97],
          [67, 57, 39, ..., 25, 50, 91]],

         [[95, 85, 50, ..., 55, 78, 57],
          [14,  7, 16, ..., 54,  7, 62],
          [54, 23, 15, ..., 39, 80, 91],
          ...,
          [81, 54,  2, ..., 76, 24, 99],
          [25, 95, 66, ..., 81, 32, 35],
          [29, 45,  1, ..., 25, 83, 20]],

         [[ 4, 59,  6, ..., 17, 69, 19],
          [82, 93,  9, ...,  5, 11, 90],
          [11, 53, 15, ..., 13, 64, 82],
          ...,
          [75, 48, 33, ..., 80, 45, 13],
          [50, 61,  4, ..., 45, 70, 74],
          [67, 36, 29, ..., 41, 58, 38]]]]])
Coordinates:
  * mode         (mode) <U4 'car' 'bike' 'foot'
  * day          (day) datetime64[ns] 2015-01-01 2015-01-02 ... 2015-04-10
  * time         (time) int64 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23
  * origin       (origin) object MULTIPOLYGON (((-81.4727554321289 36.2343559...
  * destination  (destination) object MULTIPOLYGON (((-81.4727554321289 36.23...

Set two `GeometryIndex` instances for the `origin` and `destination` coordinates, respectively (first drop the two `pandas.Index` objects that were set by default for these two dimension coordinates).

In [6]:
arr = (
    arr
    .drop_indexes(["origin", "destination"])
    .set_xindex("origin", GeometryIndex, crs=origin.crs)
    .set_xindex("destination", GeometryIndex, crs=destination.crs)
)

arr

<xarray.DataArray (mode: 3, day: 100, time: 24, origin: 100, destination: 100)>
array([[[[[93, 38,  9, ...,  4, 45, 31],
          [49, 62, 40, ...,  5,  9, 59],
          [26, 14, 86, ..., 36, 55, 70],
          ...,
          [63, 22, 82, ..., 28, 16, 27],
          [51, 52, 33, ..., 79, 51, 57],
          [34, 19, 14, ...,  6, 25, 67]],

         [[10, 48, 23, ..., 95, 97,  3],
          [67, 99, 47, ..., 26, 82, 10],
          [66, 49,  3, ..., 11, 79, 32],
          ...,
          [79, 55, 51, ..., 16, 95, 83],
          [50,  1, 75, ..., 57, 34, 30],
          [32, 44, 56, ..., 12, 42, 96]],

         [[25, 65, 47, ..., 72, 83, 55],
          [ 5, 97, 93, ..., 60,  7, 98],
          [58, 44, 38, ..., 27, 72, 14],
          ...,
...
          ...,
          [23, 19, 82, ..., 91,  4, 90],
          [ 7, 86, 98, ..., 81, 13, 97],
          [67, 57, 39, ..., 25, 50, 91]],

         [[95, 85, 50, ..., 55, 78, 57],
          [14,  7, 16, ..., 54,  7, 62],
          [54, 23, 15, ..., 39, 80, 91],
          ...,
          [81, 54,  2, ..., 76, 24, 99],
          [25, 95, 66, ..., 81, 32, 35],
          [29, 45,  1, ..., 25, 83, 20]],

         [[ 4, 59,  6, ..., 17, 69, 19],
          [82, 93,  9, ...,  5, 11, 90],
          [11, 53, 15, ..., 13, 64, 82],
          ...,
          [75, 48, 33, ..., 80, 45, 13],
          [50, 61,  4, ..., 45, 70, 74],
          [67, 36, 29, ..., 41, 58, 38]]]]])
Coordinates:
  * mode         (mode) <U4 'car' 'bike' 'foot'
  * day          (day) datetime64[ns] 2015-01-01 2015-01-02 ... 2015-04-10
  * time         (time) int64 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23
  * origin       (origin) object MULTIPOLYGON (((-81.4727554321289 36.2343559...
  * destination  (destination) object MULTIPOLYGON (((-81.4727554321289 36.23...
Indexes:
    origin       GeometryIndex(crs=EPSG:4267)
    destination  GeometryIndex(crs=EPSG:4267)

In [7]:
arr.xindexes

Indexes:
    mode         PandasIndex
    day          PandasIndex
    time         PandasIndex
    origin       GeometryIndex(crs=EPSG:4267)
    destination  GeometryIndex(crs=EPSG:4267)

Access to the CRS via the index:

In [8]:
arr.xindexes["origin"].crs

<Geographic 2D CRS: EPSG:4267>
Name: NAD27
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North and central America: Antigua and Barbuda - onshore. Bahamas - onshore plus offshore over internal continental shelf only. Belize - onshore. British Virgin Islands - onshore. Canada onshore - Alberta, British Columbia, Manitoba, New Brunswick, Newfoundland and Labrador, Northwest Territories, Nova Scotia, Nunavut, Ontario, Prince Edward Island, Quebec, Saskatchewan and Yukon - plus offshore east coast. Cuba - onshore and offshore. El Salvador - onshore. Guatemala - onshore. Honduras - onshore. Panama - onshore. Puerto Rico - onshore. Mexico - onshore plus offshore east coast. Nicaragua - onshore. United States (USA) onshore and offshore - Alabama, Alaska, Arizona, Arkansas, California, Colorado, Connecticut, Delaware, Florida, Georgia, Idaho, Illinois, Indiana, Iowa, Kansas, Kentucky, Louisiana, Maine, Maryland, 

"Strict" selection of geometries.

In [9]:
arr.sel(origin=[origin[0], origin[5]])

<xarray.DataArray (mode: 3, day: 100, time: 24, origin: 2, destination: 100)>
array([[[[[93, 38,  9, ...,  4, 45, 31],
          [13, 77, 81, ..., 92, 13, 54]],

         [[10, 48, 23, ..., 95, 97,  3],
          [30, 38, 56, ..., 84, 91, 44]],

         [[25, 65, 47, ..., 72, 83, 55],
          [90, 97, 52, ..., 16, 33, 80]],

         ...,

         [[65, 98, 32, ..., 13, 88, 39],
          [36, 70, 20, ..., 55, 91, 27]],

         [[56, 57, 14, ..., 81, 96, 11],
          [30, 12, 56, ..., 84, 63, 69]],

         [[69, 12, 89, ..., 77, 89, 70],
          [94, 93,  4, ..., 88, 26, 77]]],

...

        [[[60, 66, 58, ..., 96, 79, 16],
          [67, 14, 14, ..., 44, 85, 86]],

         [[52, 99, 50, ..., 65, 88, 48],
          [76, 38, 54, ..., 62, 23, 33]],

         [[84,  2, 57, ..., 59, 13, 10],
          [42, 88, 42, ..., 52, 36, 95]],

         ...,

         [[61, 31, 53, ..., 92, 83, 21],
          [11, 87, 17, ...,  7, 37, 32]],

         [[95, 85, 50, ..., 55, 78, 57],
          [38, 48, 13, ...,  3, 70, 19]],

         [[ 4, 59,  6, ..., 17, 69, 19],
          [90, 83, 77, ..., 88, 42, 31]]]]])
Coordinates:
  * mode         (mode) <U4 'car' 'bike' 'foot'
  * day          (day) datetime64[ns] 2015-01-01 2015-01-02 ... 2015-04-10
  * time         (time) int64 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23
  * origin       (origin) object MULTIPOLYGON (((-81.4727554321289 36.2343559...
  * destination  (destination) object MULTIPOLYGON (((-81.4727554321289 36.23...
Indexes:
    origin       GeometryIndex(crs=EPSG:4267)
    destination  GeometryIndex(crs=EPSG:4267)

Spatial-aware data selection using the "nearest" mode.

In [10]:
arr.sel(
    day="2015-01-01",
    time=0,
    origin=[shapely.Point(-80, 35), shapely.Point(-76, 35.6)],
    destination=[shapely.Point(-81, 36), shapely.Point(-77, 35.5)],
    method="nearest",
)

<xarray.DataArray (mode: 3, origin: 2, destination: 2)>
array([[[52, 62],
        [66, 29]],

       [[41, 97],
        [30, 54]],

       [[73, 10],
        [65, 86]]])
Coordinates:
  * mode         (mode) <U4 'car' 'bike' 'foot'
    day          datetime64[ns] 2015-01-01
    time         int64 0
  * origin       (origin) object MULTIPOLYGON (((-79.91995239257812 34.807918...
  * destination  (destination) object MULTIPOLYGON (((-80.72651672363281 35.5...
Indexes:
    origin       GeometryIndex(crs=EPSG:4267)
    destination  GeometryIndex(crs=EPSG:4267)

Spatial-aware data selection using the "query" mode with a single geometry and a given predicate:

In [11]:
arr.sel(origin=shapely.box(-80, 35, -77, 38), method="intersects")

<xarray.DataArray (mode: 3, day: 100, time: 24, origin: 41, destination: 100)>
array([[[[[78, 29, 75, ..., 39, 93, 97],
          [84, 21, 64, ...,  7, 67,  5],
          [20, 96, 69, ..., 51, 96, 44],
          ...,
          [ 9,  9, 51, ..., 22, 29, 87],
          [79, 63, 20, ..., 37, 85, 63],
          [60, 50, 35, ..., 67, 91, 44]],

         [[88, 85, 29, ..., 34, 38,  4],
          [94, 70, 54, ..., 24, 97, 53],
          [35, 15, 35, ..., 46, 35, 51],
          ...,
          [76, 75, 34, ..., 29, 84, 27],
          [32, 42, 60, ..., 21, 29, 53],
          [61, 76, 63, ..., 55, 11, 72]],

         [[52, 64, 89, ..., 76, 62, 18],
          [34, 25, 76, ..., 88,  7, 41],
          [37, 48, 85, ..., 81, 99, 48],
          ...,
...
          ...,
          [76, 79, 94, ..., 84, 85, 79],
          [41, 59, 54, ..., 77, 15, 37],
          [57, 75,  9, ..., 76, 73, 16]],

         [[82, 18, 39, ..., 52, 11, 70],
          [74, 20, 85, ..., 41, 14, 77],
          [42, 60, 43, ..., 10, 13,  4],
          ...,
          [84, 84, 81, ...,  7, 45, 27],
          [68, 68, 35, ..., 51, 66, 81],
          [41, 32, 97, ..., 17, 76, 14]],

         [[49,  2, 91, ..., 36, 88, 82],
          [70, 43, 16, ..., 47, 41, 37],
          [87, 26, 78, ..., 28, 67, 76],
          ...,
          [20, 92, 47, ..., 22, 65, 80],
          [15, 14, 42, ..., 76, 27, 94],
          [79,  4, 49, ..., 84,  8, 73]]]]])
Coordinates:
  * mode         (mode) <U4 'car' 'bike' 'foot'
  * day          (day) datetime64[ns] 2015-01-01 2015-01-02 ... 2015-04-10
  * time         (time) int64 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23
  * origin       (origin) object MULTIPOLYGON (((-78.11376953125 34.720985412...
  * destination  (destination) object MULTIPOLYGON (((-81.4727554321289 36.23...
Indexes:
    origin       GeometryIndex(crs=EPSG:4267)
    destination  GeometryIndex(crs=EPSG:4267)

"Query" mode selection with an array-like of geometries is not supported:

In [12]:
arr.sel(origin=[shapely.Point(-80, 35), shapely.Point(-76, 35.6)], method="intersects")

ValueError: selection with another method than nearest only supports a single geometry as input label.

Other operations like `isel`, `concat` and alignment work just like with any default (pandas) index:

In [13]:
arr + arr.isel(origin=[0, 1], destination=[-2, -1])

<xarray.DataArray (mode: 3, day: 100, time: 24, origin: 2, destination: 2)>
array([[[[[ 90,  62],
          [ 18, 118]],

         [[194,   6],
          [164,  20]],

         [[166, 110],
          [ 14, 196]],

         ...,

         [[176,  78],
          [ 32, 136]],

         [[192,  22],
          [ 62,  40]],

         [[178, 140],
          [ 80,  86]]],

...

        [[[158,  32],
          [  8, 156]],

         [[176,  96],
          [ 58, 176]],

         [[ 26,  20],
          [ 68, 152]],

         ...,

         [[166,  42],
          [ 70, 186]],

         [[156, 114],
          [ 14, 124]],

         [[138,  38],
          [ 22, 180]]]]])
Coordinates:
  * mode         (mode) <U4 'car' 'bike' 'foot'
  * day          (day) datetime64[ns] 2015-01-01 2015-01-02 ... 2015-04-10
  * time         (time) int64 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23
  * origin       (origin) object MULTIPOLYGON (((-81.4727554321289 36.2343559...
  * destination  (destination) object MULTIPOLYGON (((-77.96073150634766 34.1...
Indexes:
    origin       GeometryIndex(crs=EPSG:4267)
    destination  GeometryIndex(crs=EPSG:4267)

In [14]:
xarray.concat([arr.isel(origin=[0, 1]), arr.isel(origin=[-2, -1])], dim="origin")

<xarray.DataArray (mode: 3, day: 100, time: 24, origin: 4, destination: 100)>
array([[[[[93, 38,  9, ...,  4, 45, 31],
          [49, 62, 40, ...,  5,  9, 59],
          [51, 52, 33, ..., 79, 51, 57],
          [34, 19, 14, ...,  6, 25, 67]],

         [[10, 48, 23, ..., 95, 97,  3],
          [67, 99, 47, ..., 26, 82, 10],
          [50,  1, 75, ..., 57, 34, 30],
          [32, 44, 56, ..., 12, 42, 96]],

         [[25, 65, 47, ..., 72, 83, 55],
          [ 5, 97, 93, ..., 60,  7, 98],
          [74, 74, 53, ..., 85, 30, 84],
          [15, 12, 84, ...,  1, 33, 97]],

         ...,

         [[65, 98, 32, ..., 13, 88, 39],
          [20, 36, 74, ..., 52, 16, 68],
          [24, 53, 24, ..., 43, 32, 43],
...
          [99, 52, 72, ..., 47, 34, 76],
          [25, 38, 67, ...,  1, 76, 25],
          [50, 41, 15, ..., 28, 50, 12]],

         ...,

         [[61, 31, 53, ..., 92, 83, 21],
          [72, 96, 48, ..., 89, 35, 93],
          [ 7, 86, 98, ..., 81, 13, 97],
          [67, 57, 39, ..., 25, 50, 91]],

         [[95, 85, 50, ..., 55, 78, 57],
          [14,  7, 16, ..., 54,  7, 62],
          [25, 95, 66, ..., 81, 32, 35],
          [29, 45,  1, ..., 25, 83, 20]],

         [[ 4, 59,  6, ..., 17, 69, 19],
          [82, 93,  9, ...,  5, 11, 90],
          [50, 61,  4, ..., 45, 70, 74],
          [67, 36, 29, ..., 41, 58, 38]]]]])
Coordinates:
  * mode         (mode) <U4 'car' 'bike' 'foot'
  * day          (day) datetime64[ns] 2015-01-01 2015-01-02 ... 2015-04-10
  * time         (time) int64 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23
  * origin       (origin) object MULTIPOLYGON (((-81.4727554321289 36.2343559...
  * destination  (destination) object MULTIPOLYGON (((-81.4727554321289 36.23...
Indexes:
    origin       GeometryIndex(crs=EPSG:4267)
    destination  GeometryIndex(crs=EPSG:4267)